In [2]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [4]:
def get_token():
    load_dotenv('../.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [5]:
def get_available_bikes():
    load_dotenv('../.env')
    token = os.environ.get("access_token")
    url = f"https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

CLEANING THE NEW DATA

In [6]:
data_bikes = get_available_bikes()
bicimad_real_time = pd.DataFrame(get_available_bikes()["data"])

In [7]:
bicimad_real_time[["longitude", "latitude"]] = bicimad_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))

In [8]:
bicimad_real_time = bicimad_real_time.drop(["geofence", "integrator", "reservations_count", "tipo_estacionPBSC", "virtual_bikes", "virtual_bikes_num", "code_district", "code_suburb", "bikesGo", "geometry", "light", "no_available", "number", "virtualDelete", "geofenced_capacity"], axis = 1)


In [23]:
bicimad_real_time

,activate,address,dock_bikes,free_bases,id,name,total_bases,longitude,latitude
0,1,"Avenida del Ensanche de Vallecas, 9,",20,7,2190,"453 - Avenida del Ensanche de Vallecas, 9",27,-3.612530,40.370440
1,1,"Paseo de la Chopera,33,Comunidad de Madrid España",3,20,2205,"267 - Paseo de la Chopera, 33",23,-3.700400,40.395000
2,1,"Paseo de la Castellana nº 122,",13,10,2224,150 - Castellana frente a Hermanos Pinzón,23,-3.690800,40.449100
3,1,"Paseo de la Castellana nº 164,Comunidad de Mad...",9,14,2225,157- Castellana 164,23,-3.689415,40.459137
4,1,"Guetaria , 84b,",23,1,2226,395b - 395 - Guetaria 84b,24,-3.715691,40.369168
...,...,...,...,...,...,...,...,...,...
608,1,"Calle Segovia nº 26,",17,10,1572,166 - Segovia 26,27,-3.713377,40.413905
609,1,"Calle del Sorbe , 20,",22,2,2070,"357 - Calle del Sorbe , 20",24,-3.713209,40.389870
610,1,"Avenida del Mediterráneo nº 19,",14,10,1478,76 - Puerta de Mariano de Cavia,24,-3.675102,40.407271
611,0,"Calle Francisco Balseiro nº 1,",0,0,1643,237 - Pablo Iglesias,0,-3.710525,40.451333


GETTING THE NEAREST STATION FROM THE REAL TIME DATA

In [10]:
places = requests.get("https://datos.madrid.es/egob/catalogo/300614-0-centros-educativos.json")
places = places.json()
places = places["@graph"]
places = pd.json_normalize(places)
places = places.drop(["@id", "id", "relation", "address.district.@id", "address.area.@id", "address.locality", "address.postal-code", "organization.organization-desc", "organization.accesibility", "organization.schedule", "organization.services", "@type"], axis = 1)

In [13]:
#Global variables
places_lat_rad = np.radians(places['location.latitude'].to_numpy())
places_lon_rad = np.radians(places['location.longitude'].to_numpy())

In [14]:
def fnearest_bicimad_realtime(places, df):
    data_list = []
    df = df.loc[df["dock_bikes"] > 0]
    bicimad_real_time_lat_rad = np.radians(bicimad_real_time['latitude'].to_numpy())
    bicimad_real_time_lon_rad = np.radians(bicimad_real_time['longitude'].to_numpy())
    dlat = bicimad_real_time_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_real_time_lon_rad[:, np.newaxis] - places_lon_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_real_time_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad_real_time['name'].iloc[station_index]
        station_address = bicimad_real_time["address"].iloc[station_index]
        bikes_available = bicimad_real_time["dock_bikes"].iloc[station_index]
        station_id = bicimad_real_time["id"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"place": place, "place_address": place_address, "station_name": station, "bikes_available": bikes_available, "station_id" : station_id, "station_address": station_address,  "distance": min_distance})
        
    nearest_bicimad_realtime = pd.DataFrame(data_list)
    return nearest_bicimad_realtime

In [15]:
nearest_bicimad_realtime = fnearest_bicimad_realtime(places, bicimad_real_time)

In [16]:
nearest_bicimad_realtime

,place,place_address,station_name,bikes_available,station_id,station_address,distance
0,ABC Schule (Alemán),CALLE LEIZARAN 27,147 - Doctor Arce 45,9,1553,"Avenida del Doctor Arce nº 45,",175.64
1,Academia Darío Estudio,CALLE GRAN VIA 63,116b - Plaza de España B,2,1522,"Calle Princesa nº 5,",145.70
2,AFI Escuela de Finanzas,CALLE MARQUES DE VILLAMEJOR 5,103 - Ortega y Gasset,16,1506,"Calle José Ortega y Gasset nº 4,",105.45
3,Agencia Estatal Consejo Superior de Investigac...,CALLE SERRANO 115 BIS,250 - Serrano 113,8,1656,"Calle Serrano nº 113B,",174.10
4,Alliance Française de Madrid,CUESTA SANTO DOMINGO 13,23 - Santo Domingo,2,1426,"Plaza de Santo Domingo nº 1,",63.54
...,...,...,...,...,...,...,...
1879,Universidad Pontificia Comillas. Facultad de D...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,2,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87
1880,Universidad Pontificia Comillas. Facultad de T...,CALLE UNIVERSIDAD DE COMILLAS 3 y 5,579 - Maria Tubau,7,2256,"579 - Maria Tubau,",4342.76
1881,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,2,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87
1882,Universidad Pontificia de Comillas. Facultad d...,CALLE ALBERTO AGUILERA 23,57 - Santa Cruz del Marcenado,2,1461,"Calle Santa Cruz del Marcenado nº 24,",124.87


In [20]:
def fnearest_bicimad_realtime_parking(places, df):
    data_list = []
    df = df[df["free_bases"] > 0]
    bicimad_real_time_lat_rad = np.radians(bicimad_real_time['latitude'].to_numpy())
    bicimad_real_time_lon_rad = np.radians(bicimad_real_time['longitude'].to_numpy())
    dlat = bicimad_real_time_lat_rad[:, np.newaxis] - places_lat_rad
    dlon = bicimad_real_time_lon_rad[:, np.newaxis] - places_lon_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(bicimad_real_time_lat_rad[:, np.newaxis]) * np.cos(places_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(places["title"])):
        station_index = min_distance_indices[x]
        station = bicimad_real_time['name'].iloc[station_index]
        station_address = bicimad_real_time["address"].iloc[station_index]
        free_bases = bicimad_real_time["free_bases"].iloc[station_index]
        station_id = bicimad_real_time["id"].iloc[station_index]
        place_address = places["address.street-address"][x]
        place = places["title"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"place": place, "place_address": place_address, "station_name": station, "free_bases": free_bases, "station_id" : station_id, "station_address": station_address,  "distance": min_distance})
        
    return pd.DataFrame(data_list)

In [21]:
nearest_bicimad_realtime_parking = fnearest_bicimad_realtime_parking(places, bicimad_real_time)